In [1]:
import json
import analyses.run_apsim
import pandas as pd
import database as db
import geopandas as gpd
import numpy as np
from xml.etree import ElementTree
import analyses.run_apsim
import shutil
from analyses.munging import get_centroid
from apsim.apsim_input_writer import create_mukey_runs
from apsim.apsim_output_parser import parse_all_output_field, parse_summary_output_field

In [2]:
dbconn = db.connect_to_db('database.ini')

### Basswood

In [3]:
basswood_ssurgo = gpd.read_file("C:\\Users\\mjn\\Documents\\Foresite\\yield_monitor_data\\analyses\\strips\\basswood\\basswood_ssurgo.geojson")
basswood_mukeys = list(np.unique(basswood_ssurgo['mukey']))
print(basswood_mukeys)
interim_ssurgo = gpd.read_file("C:\\Users\\mjn\\Documents\\Foresite\\yield_monitor_data\\analyses\\strips\\interim\\interim_ssurgo.geojson")
interim_mukeys = list(np.unique(interim_ssurgo['mukey']))
print(interim_mukeys)
orbweaversouth_ssurgo = gpd.read_file("C:\\Users\\mjn\\Documents\\Foresite\\yield_monitor_data\\analyses\\strips\\orbweaver-south\\orbweaver_south_ssurgo.geojson")
orbweaversouth_mukeys = list(np.unique(orbweaversouth_ssurgo['mukey']))
print(orbweaversouth_mukeys)
orbweavernorth_ssurgo = gpd.read_file("C:\\Users\\mjn\\Documents\\Foresite\\yield_monitor_data\\analyses\\strips\\orbweaver-north\\orbweaver_north_ssurgo.geojson")
orbweavernorth_mukeys = list(np.unique(orbweavernorth_ssurgo['mukey']))
print(orbweavernorth_mukeys)

['407624', '407671', '407690', '407691', '407698', '407707']
['407623', '407624', '407636', '407649', '407650', '407651', '407671', '407690', '407703']
['407623', '407651', '407691']
['407650', '407651', '407671', '407691']


In [16]:
ssurgo_centroid = get_centroid(orbweavernorth_ssurgo, 'areasymbol', "geometry")
ssurgo_centroid

array([ 41.54982834, -93.2736912 ])

### Soybean runs 2007, 2009, 2011, 2013, 2015

In [13]:
soybean_years = [2007, 2009, 2011, 2013, 2015]
corn_years = [2008, 2010, 2012, 2014, 2018]

In [6]:
def create_and_run_strips_apsim_files(field_name, mukeys, rotation, met_name, year_list, sfc_mgmt=None, cfs_mgmt=None):
    for i in year_list:
        end_year = i
        start_year = end_year - 3
        prior_year = end_year - 1
        if rotation == 'sfc':
            sfc_mgmt = json.loads( open( f'crop_jsons/strips_sfc_{end_year}.json', 'r' ).read() )
            cfs_mgmt = json.loads( open( f'crop_jsons/strips_cfs_{prior_year}.json', 'r' ).read() )
            create_mukey_runs(mukeys, dbconn, 'sfc', f'{met_name}.met', f'{field_name}Default', start_year=start_year, end_year=end_year, sfc_mgmt=sfc_mgmt, cfs_mgmt=cfs_mgmt)
            create_mukey_runs(mukeys, dbconn, 'sfc', f'{met_name}.met', f'{field_name}Saxton', start_year=start_year, end_year=end_year, sfc_mgmt=sfc_mgmt, cfs_mgmt=cfs_mgmt, saxton=True)
            original = f'met_files/{met_name}.met'
            target = f'apsim_files/{field_name}Default/{end_year}/sfc/met_files/{met_name}.met'
            shutil.copyfile(original, target)
            target = f'apsim_files/{field_name}Saxton/{end_year}/sfc/met_files/{met_name}.met'
            shutil.copyfile(original, target)
            analyses.run_apsim.run_all_simulations(apsim_files_path=f"apsim_files\\{field_name}Default\\{end_year}\\sfc\\*.apsim", sim_files_path=f"apsim_files\\{field_name}Default\\{end_year}\\sfc\\*.sim")
            analyses.run_apsim.run_all_simulations(apsim_files_path=f"apsim_files\\{field_name}Saxton\\{end_year}\\sfc\\*.apsim", sim_files_path=f"apsim_files\\{field_name}Saxton\\{end_year}\\sfc\\*.sim")
        elif rotation == 'cfs':
            sfc_mgmt = json.loads( open( f'crop_jsons/strips_sfc_{prior_year}.json', 'r' ).read() )
            cfs_mgmt = json.loads( open( f'crop_jsons/strips_cfs_{end_year}.json', 'r' ).read() )
            create_mukey_runs(mukeys, dbconn, 'cfs', f'{met_name}.met', f'{field_name}Default', start_year=start_year, end_year=end_year, sfc_mgmt=sfc_mgmt, cfs_mgmt=cfs_mgmt)
            create_mukey_runs(mukeys, dbconn, 'cfs', f'{met_name}.met', f'{field_name}Saxton', start_year=start_year, end_year=end_year, sfc_mgmt=sfc_mgmt, cfs_mgmt=cfs_mgmt, saxton=True)
            original = f'met_files/{met_name}.met'
            target = f'apsim_files/{field_name}Default/{end_year}/cfs/met_files/{met_name}.met'
            shutil.copyfile(original, target)
            target = f'apsim_files/{field_name}Saxton/{end_year}/cfs/met_files/{met_name}.met'
            shutil.copyfile(original, target)
            analyses.run_apsim.run_all_simulations(apsim_files_path=f"apsim_files\\{field_name}Default\\{end_year}\\cfs\\*.apsim", sim_files_path=f"apsim_files\\{field_name}Default\\{end_year}\\cfs\\*.sim")
            analyses.run_apsim.run_all_simulations(apsim_files_path=f"apsim_files\\{field_name}Saxton\\{end_year}\\cfs\\*.apsim", sim_files_path=f"apsim_files\\{field_name}Saxton\\{end_year}\\cfs\\*.sim")
        else:
            print("That rotation is not currently set up")

In [14]:
create_and_run_apsim_files('OrbweaverSouth', orbweaversouth_mukeys, 'cfs', 'orbweaversouth', corn_years, sfc_mgmt=None, cfs_mgmt=None)
create_and_run_apsim_files('OrbweaverSouth', orbweaversouth_mukeys, 'sfc', 'orbweaversouth', soybean_years, sfc_mgmt=None, cfs_mgmt=None)

Finished! All files created!
Finished! All files created!
Running on 62 cores
Converting 3 .apsim files to .sim files.
Running Apsim for 3 .sim files...
Runs completed.
Processing time: 2.5577 seconds
Running on 62 cores
Converting 3 .apsim files to .sim files.
Running Apsim for 3 .sim files...
Runs completed.
Processing time: 2.5569 seconds
Finished! All files created!
Finished! All files created!
Running on 62 cores
Converting 3 .apsim files to .sim files.
Running Apsim for 3 .sim files...
Runs completed.
Processing time: 2.5582 seconds
Running on 62 cores
Converting 3 .apsim files to .sim files.
Running Apsim for 3 .sim files...
Runs completed.
Processing time: 2.5518 seconds
Finished! All files created!
Finished! All files created!
Running on 62 cores
Converting 3 .apsim files to .sim files.
Running Apsim for 3 .sim files...
Runs completed.
Processing time: 2.5567 seconds
Running on 62 cores
Converting 3 .apsim files to .sim files.
Running Apsim for 3 .sim files...
Runs completed.


In [15]:
create_and_run_apsim_files('OrbweaverNorth', orbweavernorth_mukeys, 'cfs', 'orbweavernorth', corn_years, sfc_mgmt=None, cfs_mgmt=None)
create_and_run_apsim_files('OrbweaverNorth', orbweavernorth_mukeys, 'sfc', 'orbweavernorth', soybean_years, sfc_mgmt=None, cfs_mgmt=None)

Finished! All files created!
Finished! All files created!
Running on 62 cores
Converting 4 .apsim files to .sim files.
Running Apsim for 4 .sim files...
Runs completed.
Processing time: 2.5755 seconds
Running on 62 cores
Converting 4 .apsim files to .sim files.
Running Apsim for 4 .sim files...
Runs completed.
Processing time: 2.5676 seconds
Finished! All files created!
Finished! All files created!
Running on 62 cores
Converting 4 .apsim files to .sim files.
Running Apsim for 4 .sim files...
Runs completed.
Processing time: 2.5651 seconds
Running on 62 cores
Converting 4 .apsim files to .sim files.
Running Apsim for 4 .sim files...
Runs completed.
Processing time: 2.5742 seconds
Finished! All files created!
Finished! All files created!
Running on 62 cores
Converting 4 .apsim files to .sim files.
Running Apsim for 4 .sim files...
Runs completed.
Processing time: 2.5713 seconds
Running on 62 cores
Converting 4 .apsim files to .sim files.
Running Apsim for 4 .sim files...
Runs completed.


In [ ]:
field_name = 'OrbweaverSouth'
met_name = 'orbweaversouth'

In [ ]:
startYear = 2012
priorYear = 2014
endYear = 2015

In [ ]:
sfc_mgmt = json.loads( open( f'crop_jsons/strips_sfc_{endYear}.json', 'r' ).read() )
cfs_mgmt = json.loads( open( f'crop_jsons/strips_cfs_{priorYear}.json', 'r' ).read() )

In [ ]:
print(sfc_mgmt)

In [ ]:
create_mukey_runs(mukeys, dbconn, 'sfc', f'{met_name}.met', f'{field_name}Default', start_year=startYear, end_year=endYear, sfc_mgmt=sfc_mgmt, cfs_mgmt=cfs_mgmt)
create_mukey_runs(mukeys, dbconn, 'sfc', f'{met_name}.met', f'{field_name}Saxton', start_year=startYear, end_year=endYear, sfc_mgmt=sfc_mgmt, cfs_mgmt=cfs_mgmt, saxton=True)

In [ ]:
original = f'met_files/{met_name}.met'
target = f'apsim_files/{field_name}Default/{endYear}/sfc/met_files/{met_name}.met'
shutil.copyfile(original, target)
target = f'apsim_files/{field_name}Saxton/{endYear}/sfc/met_files/{met_name}.met'
shutil.copyfile(original, target)

In [ ]:
analyses.run_apsim.run_all_simulations(apsim_files_path=f"apsim_files\\{field_name}Default\\{endYear}\\sfc\\*.apsim", sim_files_path=f"apsim_files\\{field_name}Default\\{endYear}\\sfc\\*.sim")
analyses.run_apsim.run_all_simulations(apsim_files_path=f"apsim_files\\{field_name}Saxton\\{endYear}\\sfc\\*.apsim", sim_files_path=f"apsim_files\\{field_name}Saxton\\{endYear}\\sfc\\*.sim")

### Maize runs 2008, 2010, 2012, 2014, 2018

In [ ]:
startYear = 2015
priorYear = 2017
endYear = 2018

In [ ]:
sfc_mgmt = json.loads( open( f'crop_jsons/strips_sfc_{priorYear}.json', 'r' ).read() )
cfs_mgmt = json.loads( open( f'crop_jsons/strips_cfs_{endYear}.json', 'r' ).read() )

In [ ]:
print(cfs_mgmt)

In [ ]:
create_mukey_runs(mukeys, dbconn, 'cfs', f'{met_name}.met', f'{field_name}Default', start_year=startYear, end_year=endYear, sfc_mgmt=sfc_mgmt, cfs_mgmt=cfs_mgmt)
create_mukey_runs(mukeys, dbconn, 'cfs', f'{met_name}.met', f'{field_name}Saxton', start_year=startYear, end_year=endYear, sfc_mgmt=sfc_mgmt, cfs_mgmt=cfs_mgmt, saxton=True)

In [ ]:
original = f'met_files/{met_name}.met'
target = f'apsim_files/{field_name}Default/{endYear}/cfs/met_files/{met_name}.met'
shutil.copyfile(original, target)
target = f'apsim_files/{field_name}Saxton/{endYear}/cfs/met_files/{met_name}.met'
shutil.copyfile(original, target)

In [ ]:
analyses.run_apsim.run_all_simulations(apsim_files_path=f"apsim_files\\{field_name}Default\\{endYear}\\cfs\\*.apsim", sim_files_path=f"apsim_files\\{field_name}Default\\{endYear}\\cfs\\*.sim")
analyses.run_apsim.run_all_simulations(apsim_files_path=f"apsim_files\\{field_name}Saxton\\{endYear}\\cfs\\*.apsim", sim_files_path=f"apsim_files\\{field_name}Saxton\\{endYear}\\cfs\\*.sim")